# Required assignment 8.3 Applying the KNN method to a classification problem in Python

In [43]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

## Question 1:

Write Python code to load the Caravan data set from the provided CSV file. Display the first five rows and print basic information, including the number of rows, columns and column names.

This data set comes from the ISLR library. You can find a description of the data set here: https://cran.r-project.org/web/packages/ISLR/ISLR.pdf. Download the CSV file from this GitHub repository: https://github.com/JWarmenhoven/ISLR-python.

In [44]:
###GRADED CELL
df = None
Number_of_rows: None
Number_of_columns: None
Column_names: None

# YOUR CODE HERE
#raise NotImplementedError()

# Load the dataset
df = pd.read_csv("data/Caravan.csv")

# Display the first 5 rows
print(df.head())

# Get number of rows and columns
Number_of_rows = df.shape[0]
Number_of_columns = df.shape[1]

# Get column names
Column_names = df.columns.tolist()

# Print results
print(f"Number of rows: {Number_of_rows}")
print(f"Number of columns: {Number_of_columns}")
print(f"Column names: {Column_names}")


   Unnamed: 0  MOSTYPE  MAANTHUI  MGEMOMV  MGEMLEEF  MOSHOOFD  MGODRK  MGODPR  \
0           1       33         1        3         2         8       0       5   
1           2       37         1        2         2         8       1       4   
2           3       37         1        2         2         8       0       4   
3           4        9         1        3         3         3       2       3   
4           5       40         1        4         2        10       1       4   

   MGODOV  MGODGE  ...  APERSONG  AGEZONG  AWAOREG  ABRAND  AZEILPL  APLEZIER  \
0       1       3  ...         0        0        0       1        0         0   
1       1       4  ...         0        0        0       1        0         0   
2       2       4  ...         0        0        0       1        0         0   
3       2       4  ...         0        0        0       1        0         0   
4       1       4  ...         0        0        0       1        0         0   

   AFIETS  AINBOED  ABYSTA

#### Obtain information on the target variable

The goal is to classify whether the `Purchase` variable is a 'yes' or a 'no', indicating whether a Caravan insurance policy was bought. This variable will be your target in the classification task. Take a moment to review its distribution before moving on.

In [45]:
df.Purchase.unique() # There are only instances of "Yes" and "No"

array(['No', 'Yes'], dtype=object)

In [46]:
df['Purchase'].describe() # You can see that 5,474 of the 5,822 rows have "No" target -- the target is quite imbalanced.

count     5822
unique       2
top         No
freq      5474
Name: Purchase, dtype: object

## Question 2:

In the Caravan data set, the column `Purchase` indicates whether a customer made a purchase ('Yes' or 'No'). Write Python code to calculate and display the proportion of each class ('Yes' and 'No') in the `Purchase` column.

Hint: Use the `value_counts` method with the `normalize=True` parameter.

In [47]:
###GRADED CELL
ans2 = None
# YOUR CODE HERE
#raise NotImplementedError()

# Calculate proportions of 'Yes' and 'No'
ans2 = df['Purchase'].value_counts(normalize=True)

print(ans2)

Purchase
No     0.940227
Yes    0.059773
Name: proportion, dtype: float64


## Question 3:

Standardise the predictors of the data frame (the target should have 0/1 structure) and store your output to `df_scaled`. 

Hint: Use `scaler.fit_transform(df.iloc[:,:-1])`. Avoid using the `purchase` function because it generates binary output.

In [48]:
###GRADED CELL
scaler = StandardScaler()
df_scaled = None
# YOUR CODE HERE
#raise NotImplementedError()

#scale predictors
df_scaled = pd.DataFrame(
    scaler.fit_transform(df.iloc[:, :-1]),
    columns=df.columns[:-1]
)

print(df_scaled.head())



   Unnamed: 0   MOSTYPE  MAANTHUI   MGEMOMV  MGEMLEEF  MOSHOOFD    MGODRK  \
0   -1.731753  0.680906  -0.27258  0.406697 -1.216964  0.779405 -0.694311   
1   -1.731158  0.992297  -0.27258 -0.859500 -1.216964  0.779405  0.302552   
2   -1.730563  0.992297  -0.27258 -0.859500 -1.216964  0.779405 -0.694311   
3   -1.729968 -1.187437  -0.27258  0.406697  0.010755 -0.970980  1.299414   
4   -1.729373  1.225840  -0.27258  1.672893 -1.216964  1.479559  0.302552   

     MGODPR    MGODOV    MGODGE  ...   ALEVEN  APERSONG   AGEZONG  AWAOREG  \
0  0.217444 -0.068711 -0.161816  ... -0.20291 -0.073165 -0.081055 -0.05992   
1 -0.365410 -0.068711  0.464159  ... -0.20291 -0.073165 -0.081055 -0.05992   
2 -0.365410  0.914172  0.464159  ... -0.20291 -0.073165 -0.081055 -0.05992   
3 -0.948264  0.914172  0.464159  ... -0.20291 -0.073165 -0.081055 -0.05992   
4 -0.365410 -0.068711  0.464159  ... -0.20291 -0.073165 -0.081055 -0.05992   

     ABRAND   AZEILPL  APLEZIER   AFIETS   AINBOED  ABYSTAND  
0  0.

In [49]:
#add binary target
df_scaled["Purchase"] = df["Purchase"].apply(lambda x: 1 if x == "Yes" else 0)
print(df_scaled["Purchase"])

0       0
1       0
2       0
3       0
4       0
       ..
5817    0
5818    0
5819    1
5820    0
5821    0
Name: Purchase, Length: 5822, dtype: int64


Technically, scaling the entire data frame at once isn’t ideal, as you’ll split the data into training and test sets later. For now, it's acceptable because the focus is simply on demonstrating how to use KNN. However, in practice, this approach introduces data leakage in which the test set influences the scaling process, which means information from the test data is indirectly used during training.

## Question 4:

Create a data frame that includes only the predictors and the target variable, and then sample 20 per cent of the data as a test set.

1. Use the `train_test_split` function from `sklearn.model_selection` to split `df_scaled` into training and testing sets, with 20 per cent reserved for testing.

2. From the training set, create `X_train` by dropping the `Purchase` column, and set `y_train` as the `Purchase` column.

3. Do the same for the test set: create `X_test` by dropping `Purchase`, and set `y_test` as the `Purchase` column.


In [50]:
###GRADED CELL
X_train = None
y_train = None
X_test = None
y_test = None
# YOUR CODE HERE
#raise NotImplementedError()

# Split data into train/test
train, test = train_test_split(df_scaled, test_size=0.2, random_state = 42)

# Training set
X_train = train.drop("Purchase", axis=1) #drop the purchase column
y_train = train["Purchase"] #take the purchase column

# Test set
X_test = test.drop("Purchase", axis=1)
y_test = test["Purchase"]


## Question 5:

Build a KNN classifier to predict customer purchases based on the training data you have prepared.

1. Initialise a KNN classifier with $k = 1$.

2. Train the classifier using the training features `X_train` and target labels `y_train`.

3. Use the trained model to predict the target variable on the test features `X_test`.

4. Compute the confusion matrix to compare the true test labels `y_test` and the predicted labels. 


In [51]:
###GRADED CELL
neigh = KNeighborsClassifier(n_neighbors=1) #train a KNN with K = 1
neigh.fit(X_train, y_train)
y_pred = None
cm = None
# YOUR CODE HERE
#raise NotImplementedError()

y_pred = neigh.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

print("Confusion matrix: \n [(TN     FP]\n [FN     TP)]\n",cm)


Confusion matrix: 
 [(TN     FP]
 [FN     TP)]
 [[1020   64]
 [  67   14]]


Write a function that takes a value of $k$ and returns all relevant evaluation metrics.

In [52]:
def k_stats(k, X_train, y_train, X_test, y_test):
    neigh = KNeighborsClassifier(n_neighbors=k) #train a KNN with K = 1
    neigh.fit(X_train, y_train) #fit it to our training set
    y_pred = neigh.predict(X_test) #predict the test set
    cm = confusion_matrix(y_test, y_pred)
    nr_pos = np.sum(y_pred == 1)
    true_neg = cm[0][0]
    false_pos = cm[0][1]
    false_neg = cm[1][0]
    true_pos = cm[1][1]
    misclassification_rate = (false_pos + false_neg)/np.sum(cm)
    precision = true_pos / (true_pos + false_pos) if (true_pos + false_pos) > 0 else 0.0
    sensitivity = true_pos / (true_pos + false_neg) if (true_pos + false_neg) > 0 else 0.0
    return nr_pos, true_pos, misclassification_rate, precision, sensitivity

Construct a data frame that includes the $k$ hyperparameter, and the test statistics KNN with the corresponding $k$.

In [53]:
df_knn = pd.DataFrame(columns=['K','number pos', 'true pos', 'misclassification','precision','sensitivity'])


## Question 6:

Use the `k_stats` function in a loop, and report the results obtained by the KNN method for $k = 1,3,5,7,9$.

The function `k_stats(k, X_train, y_train, X_test, y_test)`  trains a KNN classifier with $k$ neighbours and returns several evaluation metrics:

* 'nr_pos': number of positive predictions

* 'true_pos': number of true positive predictions

* 'mc': misclassification count or error

* 'pre': precision

* 'sens': sensitivity (recall)

Perform the following tasks:

1. Run a loop over five different odd values of $k$ — specifically, $k = 1, 3, 5, 7, 9$.

2. Store the results in a data frame `df_knn`.

3. Convert the columns `k`, `number pos` and `true pos` to integer type using pandas' `pd.to_numeric()` with `downcast='integer'` for memory efficiency.

In [54]:

###GRADED CELL
df_knn["K"] = None
df_knn["number pos"] = None
df_knn["true pos"] = None



# YOUR CODE HERE
#raise NotImplementedError()

# Initialize empty DataFrame
df_knn = pd.DataFrame(columns=['K', 'number pos', 'true pos', 'misclassification', 'precision', 'sensitivity'])

# Loop through k values
for k in [1, 3, 5, 7, 9]:
    nr_pos, true_pos, mis_rate, precision, sensitivity = k_stats(k, X_train, y_train, X_test, y_test)
    # Append results as a new row
    df_knn.loc[len(df_knn)] = [k, nr_pos, true_pos, mis_rate, precision, sensitivity]

# Convert certain columns to integer type
df_knn["K"] = pd.to_numeric(df_knn.K, downcast='integer')
df_knn["number pos"] = pd.to_numeric(df_knn["number pos"], downcast='integer')
df_knn["true pos"] = pd.to_numeric(df_knn["true pos"], downcast='integer')

# Display the results
print(df_knn)

   K  number pos  true pos  misclassification  precision  sensitivity
0  1          78        14           0.112446   0.179487     0.172840
1  3          25         5           0.082403   0.200000     0.061728
2  5          11         2           0.075536   0.181818     0.024691
3  7           3         1           0.070386   0.333333     0.012346
4  9           2         1           0.069528   0.500000     0.012346


In [55]:
df_knn.style.hide(axis="index")


K,number pos,true pos,misclassification,precision,sensitivity
1,78,14,0.112446,0.179487,0.172840
3,25,5,0.082403,0.200000,0.061728
5,11,2,0.075536,0.181818,0.024691
7,3,1,0.070386,0.333333,0.012346
9,2,1,0.069528,0.500000,0.012346


Here, you can observe that increasing the value of $k$ leads to a higher misclassification error. However, take a look at the following:

In [56]:
y_test.value_counts(normalize = True)

Purchase
0    0.930472
1    0.069528
Name: proportion, dtype: float64

Here, you can see that even if you always predict 'No', the misclassification rate is already 6.6 per cent, so it's not surprising that the misclassification rate drops as $k$ increases. The `df_knn` shows that the larger $k$ gets, the fewer 'Yes' predictions are made. This means that misclassification should not be the main metric here. 

For example, you might consider $k = 3$, which still has an 8.2 per cent misclassification rate but correctly identifies seven positives, compared with just one positive when $k = 9$. This is a decision where managerial insights will play a role.
